## Tutorial: Create an OpenFlow Network and Servers

This notebook demonstrates how to create an OpenFlow network and attach servers.

## Tutorial: 

#### Configure the Environment

In [1]:
import json
import os
import chi

#Config with your project and site
chi.set('project_name', 'chameleon')   # Replace with your project name
chi.set('region_name', 'CHI@UC')       # Optional, defaults to 'CHI@UC'
chi.use_site('CHI@UC')                 # Authenticate to CHI@UC site

# Tip: Name resources with your username for easier identification
username = os.getenv("USER")
prefix = username + "_OpenFlowTutorial_"
server_name = prefix+'Server'
network_name = prefix+'Net'
subnet_name = prefix+'Subnet'
router_name = prefix+'Router'
lease_name = prefix+'Lease'

#Server attributes
image_name='CC-Ubuntu20.04'
node_type="compute_skylake"
server_count=2
physical_network="physnet1"
of_controller_ip="TODO"
of_controller_port = "6653"
vswitch_name = username+"SDN"

Now using CHI@UC:
URL: https://chi.uc.chameleoncloud.org
Location: Argonne National Laboratory, Lemont, Illinois, USA
Support contact: help@chameleoncloud.org


#### Create a Lease

In [2]:
import chi.lease
from datetime import datetime, timedelta
from dateutil import tz

BLAZAR_TIME_FORMAT = '%Y-%m-%d %H:%M'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
chi.lease.add_node_reservation(reservation_list, count=server_count, node_type=node_type)
chi.lease.add_network_reservation(reservation_list, 
                                  network_name=network_name, 
                                  physical_network=physical_network,
                                  of_controller_ip=of_controller_ip, 
                                  of_controller_port=of_controller_port, 
                                  vswitch_name=vswitch_name)
chi.lease.add_fip_reservation(reservation_list, count=2)

# Create the lease
lease = chi.lease.create_lease(lease_name, 
                               reservations=reservation_list,
                               start_date=start_date,
                               end_date=end_date)
#Print the lease info
print(json.dumps(lease, indent=2))

{
  "created_at": "2022-06-29 21:10:40",
  "updated_at": "2022-06-29 21:10:42",
  "id": "79c3c32d-5181-4518-b34c-ac31f0c9d5cb",
  "name": "adamcooper_uchicago_edu_OpenFlowTutorial_Lease",
  "user_id": "53c471e0bcb88e0f94755dfc882c2f167e5d5d45da8630560b0953c586f88fd6",
  "project_id": "4140e5f9f65545dbb9f0bdc90ef68d23",
  "start_date": "2022-06-29T21:11:00.000000",
  "end_date": "2022-06-30T21:10:00.000000",
  "trust_id": "99450579022040b296b004af28bea3cb",
  "status": "PENDING",
  "degraded": false,
  "reservations": [
    {
      "created_at": "2022-06-29 21:10:40",
      "updated_at": "2022-06-29 21:10:41",
      "id": "b0cb1ef3-2f8e-4471-aeb8-3a0058c1ba82",
      "lease_id": "79c3c32d-5181-4518-b34c-ac31f0c9d5cb",
      "resource_id": "9e6c45de-bd6f-4420-b6bb-83a992616494",
      "resource_type": "physical:host",
      "status": "pending",
      "missing_resources": false,
      "resources_changed": false,
      "hypervisor_properties": "",
      "resource_properties": "[\"==\", \"$

#### Get the Reservations

In [3]:
compute_reservation_id = [reservation for reservation in lease['reservations'] if reservation['resource_type'] == 'physical:host'][0]['id']
network_reservation_id = [reservation for reservation in lease['reservations'] if reservation['resource_type'] == 'network'][0]['id']
floatingip_reservation_id = [reservation for reservation in lease['reservations'] if reservation['resource_type'] == 'virtual:floatingip'][0]['id']

print("compute_reservation_id: " + compute_reservation_id)
print("network_reservation_id: " + network_reservation_id)
print("floatingip_reservation_id: " + floatingip_reservation_id)

compute_reservation_id: b0cb1ef3-2f8e-4471-aeb8-3a0058c1ba82
network_reservation_id: f8ecdad3-0691-46ba-851d-5b218f0b4fc3
floatingip_reservation_id: efb22ba5-27d8-45b1-acde-89ffce94b036


#### Configute the Network

In [5]:
import chi.network

network = chi.network.get_network(network_name)
network_id = network['id']

print(f'Network ID: {network_id}')

RuntimeError: No networks found with name adamcooper_uchicago_edu_OpenFlowTutorial_Net

In [ ]:
subnet = chi.network.create_subnet(subnet_name, network_id)
router = chi.network.create_router(router_name, gw_network_name='public')
chi.network.add_subnet_to_router_by_name(router_name, subnet_name)

#### Start the Server

In [ ]:
import chi.server

# Create the server
server = chi.server.create_server(server_name, 
                                  reservation_id=compute_reservation_id, 
                                  network_name=network_name, 
                                  image_name=image_name)
# Wait until the server is active
chi.server.wait_for_active(server.id)

#### Associate the Floating IP

In [ ]:
floating_ip = chi.server.associate_floating_ip(server.id)

print(f'Floating IP: {floating_ip}')

#### Delete Server

In [ ]:
chi.server.delete_server(server.id)

#### De-configure Network

In [ ]:
router_id = router['id']
subnet_id = subnet['id']

try:
    result = chi.network.remove_subnet_from_router(router_id, subnet_id)
except Exception as e:
    print(f"detach_router_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_router(router_id)
except Exception as e:
    print(f"delete_router_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_subnet(subnet_id)
except Exception as e:
    print(f"delete_subnet_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_network(network_id)
except Exception as e:
    print(f"delete_network_by_name error: {str(e)}")
    pass

#### Release Lease

In [ ]:
chi.lease.delete_lease(lease['id'])